In [1]:
import pyspark
pyspark.__version__

'3.2.1'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true


23/02/26 06:07:31 WARN Utils: Your hostname, p200300c2f72c0bb59a8d46fffeb73c34 resolves to a loopback address: 127.0.1.1; using 192.168.2.160 instead (on interface wlan0)
23/02/26 06:07:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 06:07:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!ls -la

total 899552
drwxr-xr-x  4 anberlin anberlin      4096 Feb 25 08:27 .
drwxr-xr-x 10 anberlin anberlin      4096 Feb 22 05:51 ..
drwxr-xr-x  3 anberlin anberlin      4096 Feb 25 08:14 fhv
-rw-r--r--  1 anberlin anberlin 920606870 Feb 24 05:47 fhv_tripdata_2021-06.csv
-rw-r--r--  1 anberlin anberlin     61595 Feb 25 06:56 head.csv
drwxr-xr-x  2 anberlin anberlin      4096 Feb 24 05:35 .ipynb_checkpoints
-rw-r--r--  1 anberlin anberlin    442870 Feb 25 08:27 Untitled.ipynb


In [4]:


!wc -l fhv_tripdata_2021-06.csv



14961893 fhv_tripdata_2021-06.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2021-06.csv')

In [5]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true),StructField(Affiliated_base_number,StringType,true)))

In [6]:
!head -n 1001 fhv_tripdata_2021-06.csv > head.csv


In [7]:
import pandas as pd


In [8]:
df_pandas = pd.read_csv('head.csv')

In [9]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [19]:
from pyspark.sql import types

In [61]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])


In [62]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2021-06.csv')

In [63]:
df = df.repartition(12)

In [65]:
df.write.parquet('fhv/2021/06/')

23/02/25 08:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/25 08:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/25 08:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/25 08:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/25 08:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/02/25 08:14:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/25 08:14:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/25 08:14:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers


23/02/25 08:14:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [10]:
from pyspark.sql import functions as F

In [67]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02510|2021-06-01 14:06:57|2021-06-01 14:26:39|         132|         117|      N|                  null|
|              B02869|2021-06-01 16:18:58|2021-06-01 16:41:22|         198|         196|      N|                B02869|
|              B02887|2021-06-02 17:48:58|2021-06-02 18:05:18|          33|         144|      N|                B02887|
|              B02510|2021-06-03 16:12:03|2021-06-03 16:25:53|          39|          72|      N|                  null|
|              B02764|2021-06-03 12:42:14|2021-06-03 12:51:17|          28|         130|      N|                B02764|
|              B02510|2021-06-01 01:58:4

In [6]:
df = spark.read.parquet('fhv/2021/06/')

In [7]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02510|2021-06-01 14:06:57|2021-06-01 14:26:39|         132|         117|      N|                  null|
|              B02869|2021-06-01 16:18:58|2021-06-01 16:41:22|         198|         196|      N|                B02869|
|              B02887|2021-06-02 17:48:58|2021-06-02 18:05:18|          33|         144|      N|                B02887|
|              B02510|2021-06-03 16:12:03|2021-06-03 16:25:53|          39|          72|      N|                  null|
|              B02764|2021-06-03 12:42:14|2021-06-03 12:51:17|          28|         130|      N|                B02764|
|              B02510|2021-06-01 01:58:4

In [8]:
from datetime import datetime

In [12]:
answer3 = df.withColumn('pickup_day', F.date_trunc('day', df.pickup_datetime)).where(F.col('pickup_day') == datetime(2021, 6, 15)).count()


In [13]:
answer3

452470

In [14]:
from decimal import Decimal

In [15]:
df.where("dropoff_datetime is not NULL").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02510|2021-06-01 14:06:57|2021-06-01 14:26:39|         132|         117|      N|                  null|
|              B02869|2021-06-01 16:18:58|2021-06-01 16:41:22|         198|         196|      N|                B02869|
|              B02887|2021-06-02 17:48:58|2021-06-02 18:05:18|          33|         144|      N|                B02887|
|              B02510|2021-06-03 16:12:03|2021-06-03 16:25:53|          39|          72|      N|                  null|
|              B02764|2021-06-03 12:42:14|2021-06-03 12:51:17|          28|         130|      N|                B02764|
|              B02510|2021-06-01 01:58:4

In [17]:
def trip_duration_in_hours(start: datetime, end: datetime) -> float:
    duration = end - start
    duration_in_seconds = duration.total_seconds()
    return duration_in_seconds / 3600

In [20]:
trip_duration_in_hours_udf = F.udf(trip_duration_in_hours, returnType=types.FloatType())
answer4 = df.where("pickup_datetime IS NOT NULL AND dropoff_datetime IS NOT NULL")\
            .withColumn('trip_duration_in_hours', trip_duration_in_hours_udf(df.pickup_datetime, df.dropoff_datetime))\
            .select(F.max(F.col('trip_duration_in_hours')))\
            .show()

+---------------------------+
|max(trip_duration_in_hours)|
+---------------------------+
|                   66.87889|
+---------------------------+



In [37]:
zones_df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')
zones_df.where(F.col('LocationID') == 148).show()

+----------+---------+---------------+------------+
|LocationID|  Borough|           Zone|service_zone|
+----------+---------+---------------+------------+
|       148|Manhattan|Lower East Side| Yellow Zone|
+----------+---------+---------------+------------+



In [23]:
zones_df.createOrReplaceTempView("zones")

In [41]:
answer6_python = df.join(zones_df,df.PULocationID == zones_df.LocationID).groupBy(F.col('PULocationID'), F.col('Zone')).count().sort(F.col('count').desc()).take(1)

In [42]:
answer6_python

[Row(PULocationID=61, Zone='Crown Heights North', count=231279)]

In [43]:
df.registerTempTable('fhv_tripdata')

/home/anberlin/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [44]:
answer6_sql = spark.sql("""
SELECT Zone from (
SELECT 
    PULocationID, count(1) as count
    FROM fhv_tripdata
    GROUP BY 1
    ORDER BY count desc
    LIMIT 1) tmp_table, zones
    WHERE tmp_table.PULocationID = zones.LocationID
""")

In [46]:
answer6_sql.show()

+-------------------+
|               Zone|
+-------------------+
|Crown Heights North|
+-------------------+

